<a href="https://colab.research.google.com/github/vendorofdoom/ai4g-coursework-1/blob/main/AI4G_Coursework_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Resources**

https://osmnx.readthedocs.io/en/stable/osmnx.html

https://automating-gis-processes.github.io/site/notebooks/L6/retrieve_osm_data.html

https://stackoverflow.com/questions/62637344/osmnx-is-there-a-way-to-find-an-accurate-shortest-path-between-2-coordinates

https://stackoverflow.com/questions/64333794/osmnx-graph-from-point-and-geometry-information 

https://geoffboeing.com/2021/05/osmnx-v1-1-released/

https://blog.ouseful.info/2018/06/29/working-with-openstreetmap-roads-data-using-osmnx/

https://stackoverflow.com/questions/51258029/plotting-multiple-routes-with-osmnx

https://developer.ibm.com/learningpaths/data-analysis-using-python/introduction-to-geospatial-data-using-python/


In [ ]:
!pip install osmnx

After pip installing above, you will need to restart the runtime.

In [2]:
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import math

%matplotlib inline


In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [7]:
class OSMLocation:

  def __init__(self, lat, lng, dst):

    self.lat = lat
    self.lng = lng

    self.graph = ox.graph.graph_from_point([lat, lng], dist=dst, simplify=True, network_type="walk")
    self.nodes, self.edges = ox.graph_to_gdfs(self.graph)
    
    # Get geometries for building and park footprints
    self.buildings = ox.geometries.geometries_from_point([lat, lng], dist=dst, tags={"building":True})
    self.buildings = self.buildings[self.buildings.geom_type == 'Polygon'] # keep only polygons

    self.parks = ox.geometries.geometries_from_point([lat, lng], dist=dst, tags={"landuse":"recreation_ground", "leisure": ["nature_reserve", "pitch", "playground"]})

    self.building_names = self.buildings.name.unique()
  
  def coords_for_building_name(self, building_name):
    polygon = self.buildings[self.buildings.name == building_name].iloc[0]['geometry']
    return polygon.centroid.coords[0]


  def nearest_node_for_building_name(self, building_name):

    if (building_name not in self.building_names):
      raise Exception(f"{building_name} not in graph") # TODO: raise less generic exception

    building_coords = self.coords_for_building_name(building_name)
    return ox.distance.nearest_nodes(self.graph, building_coords[0], building_coords[1])

  def node_dist(self, node1, node2):
    return ox.distance.great_circle_vec(self.nodes.loc[node1].x, 
                                        self.nodes.loc[node1].y, 
                                        self.nodes.loc[node2].x, 
                                        self.nodes.loc[node2].y) 

  def Dijkstra(self, origin, destination, max_iter=None):
    
    # code adapted from lecture notes 

    distances = {vertex: (math.inf if vertex != origin else 0) for vertex in self.graph.nodes}
    predecessors = {vertex: -1 for vertex in self.graph.nodes}

    closed = []
    fringe = list(self.graph.nodes)

    num_iter = 0

    node = None

    while fringe and (num_iter < max_iter):

      fringe_dists = {key:value for (key,value) in distances.items() if key in fringe}
      node = min(fringe_dists, key=fringe_dists.get)

      closed += [node]
      fringe.remove(node)

      successors = list(self.graph.successors(node))
      successors = [s for s in successors if s not in closed]

      for s in successors:
        if distances[s] > (distances[node] + self.graph.edges[(node, s, 0)]["length"]):
          distances[s] = (distances[node] + self.graph.edges[(node, s, 0)]["length"])
          predecessors[s] = node
        
      if node == destination:
        break

      num_iter += 1
    
    # calc shortest path
    target = destination
    path = []
    while target and predecessors[target] != -1:
      path += [target]
      target = predecessors[target]

    path += [origin]
    path.reverse()

    return [path], fringe, closed, num_iter

        
  def A_star(self, origin, destination, max_iter=None):
    
    # code adapted from lecture notes 

    closed = []
    fringe = [origin]

    g = {vertex: (math.inf if vertex != origin else 0) for vertex in self.graph.nodes}
    f = {}
    f[origin] = g[origin] + self.node_dist(origin, destination)

    predecessors = {vertex: -1 for vertex in self.graph.nodes}

    num_iter = 0

    while fringe and (num_iter < max_iter):

      f_in_fringe = {key:value for (key,value) in f.items() if key in fringe}
      node = min(f_in_fringe, key=f_in_fringe.get)

      if node == destination:
        break

      closed += [node]
      fringe.remove(node)

      successors = list(self.graph.successors(node))
      successors = [s for s in successors if s not in closed]

      for s in successors:

        g_prime = (g[node] + self.graph.edges[(node, s, 0)]["length"])

        if (s not in fringe) or (g[s] > g_prime):
          g[s] = g_prime
          f[s] = g[s] + self.node_dist(s, destination)
          predecessors[s] = node
          if (s not in fringe):
            fringe += [s]

      num_iter += 1

    # calc shortest path
    target = destination
    path = []
    while target and predecessors[target] != -1:
      path += [target]
      target = predecessors[target]

    path += [origin]
    path.reverse()

    return [path], fringe, closed, num_iter


  def other(self):
    # IDEAS
    # Based on length
    # Based on scenery
    # Based on "traffic"
    # Based on road type?
    pass

  def BFS(self, origin, destination, max_iter=None):
    
    # code adapted from lecture notes 
    
    fringe = [origin]
    closed = []

    paths_in_tree = [[origin]]

    num_iter = 0

    while fringe and (num_iter < max_iter):

      node = fringe[0]

      paths_to_node = [p for p in paths_in_tree if p[-1] == node]

      for p in paths_to_node:
        paths_in_tree.remove(p)

      if node == destination:
        return paths_to_node, fringe, closed, num_iter
      else:
        closed = closed + [node]
        fringe = fringe[1:]
        successors = list(self.graph.successors(node))

        fringe = fringe + [s for s in successors if s not in closed and s not in fringe]

        for s in successors:
          if s not in closed:
            for p in paths_to_node:
              paths_in_tree.append(p + [s])

      num_iter += 1

    return None, fringe, closed, num_iter


  def draw_map(self):
  
    fig, ax = plt.subplots(figsize=(15, 15))
    
    self.edges.plot(ax=ax, linewidth=1, edgecolor='dimgrey')
    self.nodes.plot(ax=ax, color='dimgrey', markersize=10)
    self.buildings.plot(ax=ax, facecolor='bisque')
    self.parks.plot(ax=ax, color='yellowgreen')

    return fig, ax


  def draw_path(self, origin_name, destination_name,
                path_planning_function, max_iters):

    # get coords of nodes nearest to origin and destination buildings
    origin = self.nearest_node_for_building_name(origin_name)
    destination = self.nearest_node_for_building_name(destination_name)

    # run path planning algorithm
    path, fringe, closed, total_iter = path_planning_function(origin, destination, max_iters)
    print(f"{path_planning_function.__name__} total iterations: {total_iter}")

    # draw the base of the map, i.e. building footprints and route network
    fig, ax = self.draw_map()

    # highlight origin and destination buildings on visual
    self.buildings[self.buildings.name == origin_name].plot(ax=ax, facecolor='gold')
    self.buildings[self.buildings.name == destination_name].plot(ax=ax, facecolor='gold')

    # combine paths and configure visualisation colours
    routes = []
    route_colours = []

    # found paths!
    if path:
      routes += path
      route_colours += ['black']*len(path)

    # plot fridge nodes, i.e. those that have the potential to be explored
    if fringe:
      fringe = self.nodes.loc[fringe]
      fringe.plot(ax=ax, color='mediumseagreen')
    
    # plot closed nodes, i.e. those that have been fully explored
    if closed:
      closed = self.nodes.loc[closed]
      closed.plot(ax=ax, color='tomato')

    # plot routes on map
    if len(routes) > 1:
      ox.plot_graph_routes(self.graph, routes, ax=ax, 
                           route_colors=route_colours, close=False)
      

    elif len(routes) == 1 and routes[0]:
      ox.plot_graph_route(self.graph, routes[0], ax=ax, 
                          route_color=route_colours[0], close=False)



In [8]:

def goldsmiths_map_widget(x):

  goldsmiths = OSMLocation(51.473925855158974, -0.03701156201484357, 300)

  goldsmiths.draw_path("St James Hatcham Building", "The New Cross House", goldsmiths.A_star, x)

  goldsmiths.draw_path("St James Hatcham Building", "The New Cross House", goldsmiths.Dijkstra, x)

  goldsmiths.draw_path("St James Hatcham Building", "The New Cross House", goldsmiths.BFS, x)


In [6]:
interact(goldsmiths_map_widget, x=widgets.IntSlider(min=0, max=40, step=1, value=40))

interactive(children=(IntSlider(value=40, description='x', max=40), Output()), _dom_classes=('widget-interact'…

<function __main__.goldsmiths_map_widget>